In [52]:
CREATE SCHEMA academy

Commands completed successfully.

Total execution time: 00:00:00.032

In [53]:
CREATE TABLE academy.student_performance(
	id INT NOT NULL PRIMARY KEY,
	name VARCHAR(50) NULL,
	math_score INT NULL,
    reading_score INT NULL,
    writing_score INT NULL)

Commands completed successfully.

Total execution time: 00:00:00.039

### These rows are inserted without change data capture (CDC) enabled

In [54]:
INSERT INTO academy.student_performance VALUES 
(1, 'Adam ', 80, 77, 76 ),
(2, 'Marvin', 56, 78, 59),
(3, 'Norma', 87, 78, 69),
(4, 'Ethan', 76, 69, 91)

(4 rows affected)

Total execution time: 00:00:00.042

In [55]:
SELECT * FROM academy.student_performance

(4 rows affected)

Total execution time: 00:00:00.033

id,name,math_score,reading_score,writing_score
1,Adam,80,77,76
2,Marvin,56,78,59
3,Norma,87,78,69
4,Ethan,76,69,91


CDC first needs to be enabled at the database level before it can be applied to its tables. First we check whether this is enabled for the DB and the table - it's disabled by default.

In [56]:
SELECT name, is_cdc_enabled
FROM sys.databases
WHERE name = 'sql_database'

(1 row affected)

Total execution time: 00:00:00.033

name,is_cdc_enabled
sql_database,0


In [57]:
SELECT  name, type, type_desc, is_tracked_by_cdc
FROM sys.tables
WHERE name = 'student_performance'

(1 row affected)

Total execution time: 00:00:00.185

name,type,type_desc,is_tracked_by_cdc
student_performance,U,USER_TABLE,0


### There is no such table at this point, so we get an error message

In [58]:
SELECT * FROM cdc.change_tables

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'cdc.change_tables'.

Total execution time: 00:00:00.032

In [59]:
EXEC sys.sp_cdc_enable_db

Commands completed successfully.

Total execution time: 00:00:00.489

### To enable CDC for a table, it has to be enabled for the database first

In [60]:
SELECT name, is_cdc_enabled
FROM sys.databases
WHERE name = 'sql_database'

(1 row affected)

Total execution time: 00:00:00.033

name,is_cdc_enabled
sql_database,1


### CDC needs to be explicitly turned on for each table as well

In [61]:
SELECT  name, type, type_desc, is_tracked_by_cdc
FROM sys.tables
WHERE name = 'student_performance'

(1 row affected)

Total execution time: 00:00:00.044

name,type,type_desc,is_tracked_by_cdc
student_performance,U,USER_TABLE,0


In [62]:
EXEC sys.sp_cdc_enable_table  
@source_schema = N'academy',  
@source_name   = N'student_performance',  
@role_name     = NULL,  
@supports_net_changes = 1

Commands completed successfully.

Total execution time: 00:00:00.471

In [63]:
SELECT  name, type, type_desc, is_tracked_by_cdc
FROM sys.tables
WHERE name = 'student_performance'

(1 row affected)

Total execution time: 00:00:00.045

name,type,type_desc,is_tracked_by_cdc
student_performance,U,USER_TABLE,1


### Make note of the capture\_instance here - this is what we will use to get the Log Sequence Number (LSN) corresponding to our table.

In [64]:
SELECT * FROM cdc.change_tables

(1 row affected)

Total execution time: 00:00:00.044

object_id,version,source_object_id,capture_instance,start_lsn,end_lsn,supports_net_changes,has_drop_pending,role_name,index_name,filegroup_name,create_date,partition_switch
14623095,0,1778105375,academy_student_performance,0x00000035000003980095,NULL,1,NULL,NULL,PK__student___3213E83FB4A49429,NULL,2022-11-03 09:02:36.140,1


In [65]:
INSERT INTO academy.student_performance 
VALUES (5, 'Pamela', 69, 91, 75)

(1 row affected)

Total execution time: 00:00:00.046

### Retrieve the changes recorded by invoking the cdc\_all\_changes function corresponding to our table:

[https://learn.microsoft.com/en-us/sql/relational-databases/system-functions/cdc-fn-cdc-get-all-changes-capture-instance-transact-sql](https://learn.microsoft.com/en-us/sql/relational-databases/system-functions/cdc-fn-cdc-get-all-changes-capture-instance-transact-sql)

In [66]:
DECLARE @from_lsn binary(10), @to_lsn binary(10);  
SET @from_lsn = sys.fn_cdc_get_min_lsn('academy_student_performance');  
SET @to_lsn   = sys.fn_cdc_get_max_lsn();  
SELECT * FROM cdc.fn_cdc_get_all_changes_academy_student_performance  
  (@from_lsn, @to_lsn, N'all'); 

(1 row affected)

Total execution time: 00:00:00.087

__$start_lsn,__$seqval,__$operation,__$update_mask,id,name,math_score,reading_score,writing_score
0x0000003500000DE00005,0x0000003500000DE00004,2,0x1F,5,Pamela,69,91,75


In [67]:
INSERT INTO academy.student_performance 
VALUES (6, 'Wendy', 91, 83, 94)

UPDATE academy.student_performance
SET Math_Score = 90
WHERE ID = 3

DELETE from academy.student_performance
WHERE ID = 1

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.054

In [69]:
DECLARE @from_lsn binary(10), @to_lsn binary(10);  
SET @from_lsn = sys.fn_cdc_get_min_lsn('academy_student_performance');  
SET @to_lsn   = sys.fn_cdc_get_max_lsn();  
SELECT * FROM cdc.fn_cdc_get_all_changes_academy_student_performance  
  (@from_lsn, @to_lsn, N'all')

(4 rows affected)

Total execution time: 00:00:00.044

__$start_lsn,__$seqval,__$operation,__$update_mask,id,name,math_score,reading_score,writing_score
0x0000003500000DE00005,0x0000003500000DE00004,2,0x1F,5,Pamela,69,91,75
0x0000003500000FF80004,0x0000003500000FF80003,2,0x1F,6,Wendy,91,83,94
0x00000035000010000005,0x00000035000010000004,4,0x04,3,Norma,90,78,69
0x00000035000010080005,0x00000035000010080002,1,0x1F,1,Adam,80,77,76


### Note that out of the rows still remaining in the table, IDs 3, 6, and 5 have a change record in the CDC change table.

### Let's remind ourselves of the contents of the source table at this point

In [70]:
SELECT * FROM academy.student_performance

(5 rows affected)

Total execution time: 00:00:00.049

id,name,math_score,reading_score,writing_score
2,Marvin,56,78,59
3,Norma,90,78,69
4,Ethan,76,69,91
5,Pamela,69,91,75
6,Wendy,91,83,94


### We update the students with IDs 2, 3, and 5. Both 3 and 5 already have changes recorded in the change table

In [71]:
UPDATE academy.student_performance
SET Math_Score = 61
WHERE ID = 2

UPDATE academy.student_performance
SET Math_Score = 93
WHERE ID = 3

UPDATE academy.student_performance
SET Math_Score = 67
WHERE ID = 5

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.058

### 3 new rows have been added to the all\_changes table

In [72]:
DECLARE @from_lsn binary(10), @to_lsn binary(10);  
SET @from_lsn = sys.fn_cdc_get_min_lsn('academy_student_performance');  
SET @to_lsn   = sys.fn_cdc_get_max_lsn();  
SELECT * FROM cdc.fn_cdc_get_all_changes_academy_student_performance  
  (@from_lsn, @to_lsn, N'all')

(4 rows affected)

Total execution time: 00:00:00.036

__$start_lsn,__$seqval,__$operation,__$update_mask,id,name,math_score,reading_score,writing_score
0x0000003500000DE00005,0x0000003500000DE00004,2,0x1F,5,Pamela,69,91,75
0x0000003500000FF80004,0x0000003500000FF80003,2,0x1F,6,Wendy,91,83,94
0x00000035000010000005,0x00000035000010000004,4,0x04,3,Norma,90,78,69
0x00000035000010080005,0x00000035000010080002,1,0x1F,1,Adam,80,77,76


### The fn\_cdc\_get\_net\_changes invocation returns just one entry for each row in the source.

[https://learn.microsoft.com/en-us/sql/relational-databases/system-functions/cdc-fn-cdc-get-net-changes-capture-instance-transact-sql](https://learn.microsoft.com/en-us/sql/relational-databases/system-functions/cdc-fn-cdc-get-net-changes-capture-instance-transact-sql)

Quoting from the above source: <span style="color: rgb(230, 230, 230); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px;"><i>When a source row has multiple changes during the LSN range, a single row that reflects the final content of the row is returned by the enumeration function</i></span>

In [73]:
DECLARE @from_lsn binary(10), @to_lsn binary(10);  
SET @from_lsn = sys.fn_cdc_get_min_lsn('academy_student_performance');  
SET @to_lsn   = sys.fn_cdc_get_max_lsn();  
SELECT * FROM cdc.fn_cdc_get_net_changes_academy_student_performance  
  (@from_lsn, @to_lsn, N'all')

(5 rows affected)

Total execution time: 00:00:00.099

__$start_lsn,__$operation,__$update_mask,id,name,math_score,reading_score,writing_score
0x0000003500000FF80004,2,NULL,6,Wendy,91,83,94
0x00000035000010080005,1,NULL,1,Adam,80,77,76
0x00000035000011180004,4,NULL,2,Marvin,61,78,59
0x00000035000011200003,4,NULL,3,Norma,93,78,69
0x00000035000011280003,2,NULL,5,Pamela,67,91,75


In [74]:
EXECUTE sys.sp_cdc_help_change_data_capture   
    @source_schema = N'academy',   
    @source_name = N'student_performance'; 

Commands completed successfully.

Total execution time: 00:00:00.087

source_schema,source_table,capture_instance,object_id,source_object_id,start_lsn,end_lsn,supports_net_changes,has_drop_pending,role_name,index_name,filegroup_name,create_date,index_column_list,captured_column_list
academy,student_performance,academy_student_performance,14623095,1778105375,0x00000035000003980095,NULL,1,NULL,NULL,PK__student___3213E83FB4A49429,NULL,2022-11-03 09:02:36.140,[id],"[id], [name], [math_score], [reading_score], [writing_score]"


In [75]:
EXEC sys.sp_cdc_help_jobs;

Commands completed successfully.

Total execution time: 00:00:00.029

job_type,maxtrans,maxscans,continuous,pollinginterval,retention,threshold
capture,500,10,0,0,0,0
cleanup,0,0,0,0,4320,5000


In [76]:
EXEC sys.sp_cdc_disable_db 

Commands completed successfully.

Total execution time: 00:00:00.529

In [77]:
SELECT name, is_cdc_enabled
FROM sys.databases
WHERE name = 'sql_database'

(1 row affected)

Total execution time: 00:00:00.050

name,is_cdc_enabled
sql_database,0


In [78]:
SELECT  name, type, type_desc, is_tracked_by_cdc
FROM sys.tables
WHERE name = 'student_performance'

(1 row affected)

Total execution time: 00:00:00.060

name,type,type_desc,is_tracked_by_cdc
student_performance,U,USER_TABLE,0


In [79]:
SELECT * FROM cdc.change_tables

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'cdc.change_tables'.

Total execution time: 00:00:00.031

In [80]:
DROP TABLE academy.student_performance;

Commands completed successfully.

Total execution time: 00:00:00.038

In [81]:
DROP SCHEMA academy;

Commands completed successfully.

Total execution time: 00:00:00.033